# ⛈️ ⛈️ Control of stormwater systems using pystorms ⛈️ ⛈️

pystorms is a curated collection of stormwater networks developed for prototyping and evaluating stormwater control algorithms. It uses pyswmm as its backend for interacting with the EPA-SWMM's simulation engine.

In [ ]:
# Google Colab might need to reinstall pystorms
!pip install pystorms

In [1]:
import pystorms
import matplotlib.pyplot as plt

In [2]:
# RC parms for pertty plots
plt.rcParams.update({'font.size': 14})
plt.style.use('seaborn-whitegrid')
plt.style.use('seaborn-dark-palette')

In this example, we would be using scenario theta

![theta](./theta_sheet.png)

### Model IDs in Scenario Theta
```yaml
# name of scearnio 
name: theta
# state definitions
states:
        - !!python/tuple 
          - P1
          - depthN
        - !!python/tuple
          - P2
          - depthN
# Action space 
action_space:
        - "1"
        - "2"
# Performance Targets
performance_targets:
        - !!python/tuple
          - "8"
          - flow
        - !!python/tuple
          - P1
          - flooding
        - !!python/tuple
          - P2
          - flooding
```

### Simulating control using pystorms

In [47]:
env = pystorms.scenarios.theta()
done = False


 o  Retrieving project data

pystorms abstacts the control of stormwater networks as scenarios. Each scenario comprises of a stormwater network, an event driver, a set of states and controllable assets, and a performance metric.

```python
env = pystorms.scenarios.theta()
```

The above command initalizes the stormwater control scenario and starts the simulation.

In [48]:
while not done:
    # query the current state of the simulation
    state = env.state()
    
    # Initialize random actions
    actions = [1.00, 1.00]
    
    # set the actions and progress the simulation
    done = env.step(actions)

pystorms also computes the performance metric as it runs through the simulation.

In [49]:
env.performance()

1630.3422288715237

In [19]:
env.data_log.keys()

dict_keys(['performance_measure', 'flow', 'flooding', 'simulation_time'])

In [22]:
env.data_log['flow'].keys()

dict_keys(['8'])

In [25]:
env.data_log['flow']['8'][-10:]

[4.4933228890095494e-05,
 4.492085410367944e-05,
 4.490848420155321e-05,
 4.4896119169411195e-05,
 4.4883759016013713e-05,
 4.487140375564894e-05,
 4.485905339890831e-05,
 4.484670795361076e-05,
 4.483436742480517e-05,
 4.482484695924919e-05]

### Pro Tip
data_log is structured so that you can easily export it into pandas for post-processing. 

In [17]:
import pandas as pd
flows = pd.DataFrame(index=env.data_log['simulation_time'], data=env.data_log['flow'])
flooding = pd.DataFrame(index=env.data_log['simulation_time'], data=env.data_log['flooding'])

In [18]:
flooding

,P1,P2
2018-02-25 00:00:01,0.0,0.0
2018-02-25 00:00:31,0.0,0.0
2018-02-25 00:01:01,0.0,0.0
2018-02-25 00:01:31,0.0,0.0
2018-02-25 00:02:01,0.0,0.0
...,...,...
2018-02-28 05:58:14,0.0,0.0
2018-02-28 05:58:44,0.0,0.0
2018-02-28 05:59:14,0.0,0.0
2018-02-28 05:59:44,0.0,0.0


### Exercise 1
1. Implement a rule based controller for scenario theta that uses the state information. In this sceanario states are the depths in the basins P1 and P2 at every timestep.

In [ ]:
env = pystorms.scenarios.theta()
done = False

while not done:
    # query the current state of the simulation
    state = env.state()
    
    # your control logic goes here
    
    
    # set the actions and progress the simulation
    done = env.step()

2. Plot the controlled and uncontrolled flows going into the outlet.

Note: outlet flows are logged in `env.data_log`

### Exercise 2

Implement a rule based to controller that achieves a performance of less that 1000. Peformance metric in scenario theta is computed based on the flooding and flows going into the outlet.
Uncontrolled performance(i.e., when the valves are completly open) is 1630.34

Tip 💡: Make sure your rule-based controller is not causing flooding in the basins as it tries to achive the desired performance.

### Bonus Question 💁

Implement a controller that achieves a performance metric of less than 500.